#### CIE4604 Simulation and Visualization
# Assignment 2 - GNSS-Reflectometry simulation

GPS and other GNSS satellite signals are reflected by the Earth’s oceans. A GNSS-R(eflectrometry) receiver on board a satellite in a low Earth orbit (LEO) is able to receive both the **direct** signal from a GNSS satellite, as well as the signal **reflected** by the surface of the Earth. This will provide very useful information on the Sea state and level in particular

![Remote Sensing geometry of GNSS-R (From: Jales, 2012](Jales2012.png)

For this assignment you are given the task to simulate a GNSS-R(eflectometry) experiment on board the International Space Station (ISS). 

All the Python scripts can be found in `CIE4604-M2-python` that you can download from Brightspace. For the assignment you need the following modules
- `tleplot.py`
- `crsutil.py`
- `specularpoint.py`

You will also need to have `simplekml` installed (See exercise 6). 

The `CIE4604-M2-python` download comes with 8 exercises in Jupyter notebooks. We suggest that you study the Jupyter notebooks first and do the exercises. These already contain a lot of useful material for the assignment, some of which can be re-purposed for the assignment. An other option is to do this assignment in parallel with the exercises.

## Description of the GNSS-R experiment on board the ISS

For the experiment a GNSS-R receiver will be installed on board the ISS. A GNSS-R receiver is able to

1.	Receive the direct signal from Global Navigation Satellite Systems (GNSS), such as the US GPS, Russian GLONASS, European GALILEO and Chinese BEIDOU satellites,
2.	Receive reflected GNSS signals from the Earth surface.

The path delay difference between direct and reflected signal is a measure for the Sea-surface height. However, by recording the full reflected wave form also other parameters about the sea state, such as wave-height and direction, can be retrieved.

The GNSS-R receiver on board the ISS uses two antenna’s, one **upward looking** standard **omnidirectional** antenna for the **direct** signals, and a specially designed **downward looking** antenna for the **reflected** signals. The opening angle of downward antenna (field of view) is not yet decided: the final value will depend on the outcome of the simulation, and will be a trade-off between received power and coverage.

## Breakdown in sub-tasks

The first step of any task is to break it down in more manageable sub-tasks. Normally this is something that you have to do yourself, but here, for this assignment, we will give you the task breakdown. The sub-tasks you have to perform are:

1. Simulate and visualize the ISS orbit trajectory
2. Simulate and visualize the GPS satellite orbits
3. Simulate and visualize GNSS signals from the upward looking omnidirectional antenna on the ISS (direct signals)
4. Simulate and visualize GNSS signals from the down looking antenna on the ISS (reflected signals)

More details on each sub-task can be found in the following cells.

## 1. Simulate and visualize the ISS orbit trajectory


In [ ]:
import matplotlib.pyplot as plt
import tleplot as tle
import crsutil as crs
import specularpoint as sp
import numpy as np
from scipy.io import loadmat


In [ ]:
def scircle1(lat0,lon0,theta):
    
    slat0 = np.sin(np.deg2rad(lat0))
    clat0 = np.cos(np.deg2rad(lat0))

    stheta = np.sin(np.deg2rad(theta))
    ctheta= np.cos(np.deg2rad(theta))
    
    alpha = np.deg2rad( np.arange(0,365,10) )
    
    lat = np.arctan2( slat0*ctheta + clat0*stheta*np.cos(alpha) ,
         np.sqrt( (clat0*ctheta - slat0*stheta*np.cos(alpha))**2 + (stheta *np.sin(alpha))**2 ))
    dlon = np.arctan2( stheta * np.sin(alpha) , clat0*ctheta - slat0*stheta*np.cos(alpha) )

    lat = np.rad2deg(lat)
    lon = lon0 + np.rad2deg(dlon)

    return lat, lon

Simulate and visualize the orbit trajectory of the International Space Station ISS. You may use Two-Line Elements (TLE) for ISS (Zarya) that you can find on Celestrak. The questions need to be answered in this sub-task:

a.	What are the main limitations of using GNSS-R from the ISS for the spatial and temporal coverage?
Hint: To answer this question you have to plot the ISS ground tracks (sub-satellite tracks), for which you may use `pltgroundtracks` from the `tleplot` Python module. 


In [ ]:
# get the most recent 
tle.tleget('stations')
tleERS = tle.tleread('stations.txt',verbose=0)

In [ ]:
isat,name = tle.tlefind(tleERS,'ISS (ZA')
numdays = 2
t = tle.tledatenum(['2021-11-28 0:00:00', numdays*24*60 ,0.1])
iss_xvals,iss_vvals = tle.tle2vec(tleERS,t,'ISS (ZA')
iss_xvalse,iss_vvalse = crs.eci2ecef(t,xsat=iss_xvals,vsat=iss_vvals)


plt.figure(figsize=(20,10))
coast = loadmat('coast.mat')

# using code from the Tleplot

lsat = np.arctan2(iss_xvalse[:,1], iss_xvalse[:,0]) * 180 / np.pi
dsat = np.arctan(iss_xvalse[:,2] / np.sqrt(iss_xvalse[:,1]**2 + iss_xvalse[:,0]**2) )  * 180 / np.pi

# implement masking as in TLEplot function 
lsatmask = np.ma.array(lsat)
dsatmask = np.ma.array(dsat)
lsatmask[np.hstack([False,np.abs(np.diff(lsat)) > 180])] = np.ma.masked 
dsatmask[np.hstack([False,np.abs(np.diff(dsat)) > 90])] = np.ma.masked 

Re = 6378136 # radius of earth in metres
# satetlite radius using pythagorean theorum in 3d
sat_rad = np.sqrt(np.sum(iss_xvalse**2,axis=1))


# off nadir angles in radians
nadir40 = np.deg2rad(40)/2
nadir100 = np.deg2rad(100)/2
# set up an empty array
theta40 = np.zeros_like(lsatmask)
theta100 = np.zeros_like(lsatmask)

# calculation the theta for each position
# equation from ex. 7: $\theta=\arcsin(\frac{r}{R_e}\sin(\eta))-\eta$ 
theta40 = np.rad2deg(np.arcsin((sat_rad/Re)*np.sin(nadir40)-nadir40))
theta100 = np.rad2deg(np.arcsin((sat_rad/Re)*np.sin(nadir100)-nadir100))



In [ ]:
# manually plot the circles for 40
plt.figure(figsize=(20,10))
for l,d,theta in zip(lsatmask,dsatmask,theta40):
    
    sc_lat,sc_lon = scircle1(d,l,theta)
    plt.plot(sc_lon,sc_lat,color='b',)
    
    
plt.title(f'{numdays} days of ISS Zayra Overpasses with Opening angle of 40 degrees')
plt.plot(coast["long"], coast["lat"], color='black', linewidth=0.5)
plt.xlabel("Longitude")
plt.ylabel("Latitude")


In [ ]:
# manually plot the circles for 100
plt.figure(figsize=(20,10))
for l,d,theta in zip(lsatmask,dsatmask,theta100):
    
    sc_lat,sc_lon = scircle1(d,l,theta)
    plt.plot(sc_lon,sc_lat,color='b',)
    
    
plt.title(f'{numdays} days of ISS Zayra Overpasses with Opening angle of 100 degrees')
plt.plot(coast["long"], coast["lat"], color='black', linewidth=0.5)
plt.xlabel("Longitude")
plt.ylabel("Latitude")

In [ ]:
from scipy.integrate import quad

function = lambda omega: 2*np.pi*Re*np.cos(np.deg2rad(omega))

area = quad(function,-51.64,51.64)
area_tot = quad(function,-90,90)


print(area_tot[0],'actual:',4*np.pi*Re**2)

c.	Over how many hours or days do you have to carry out the simulation so that all areas, that can possibly be observed, are observed?  

In all steps we expect clear answers with a brief explanation. Include appropriate plots and visualizations wherever necessary.

## 2. Simulate and visualize the GPS satellite orbits

Simulate and visualize the GPS orbits. You may again use TLE from Celestrak. Deliverables for this sub-task are:

a.	Plot of the GPS ground tracks.  You may again use `pltgroundtracks` from the `tleplot` module. Please think carefully about how to plot this data in a nice and concise manner, as there are about 30 operational GPS satellites.


In [ ]:
# enter here your code and run the cell to produce the result, if you wish you can use more cells
tle.tleget('GPS')
gpsERS = tle.tleread('GPS.txt',verbose=0)


In [ ]:

t = tle.tledatenum(['2021-11-23 0:00:00', 24*60 ,1])
xsat,vsat = tle.tle2vec(gpsERS,t,'GPS BIIR-2')
xsate,vsate = crs.eci2ecef(t,xsat=xsat,vsat=vsat)
plt.figure(figsize=(20,10))
tle.pltgroundtrack(xsate)


In [ ]:
# make a 3d plot
plt.clf()
fig = plt.figure("3D satellite orbit (ECI)",figsize=(20,10), tight_layout=True)
ax = fig.add_subplot(121, projection='3d')
for sat in gpsERS:
    t = tle.tledatenum(['2021-11-15 0:00:00', 24*60 ,1])
    xsat,vsat = tle.tle2vec(gpsERS,t,sat.name)
    xsate,vsate = crs.eci2ecef(t,xsat=xsat,vsat=vsat)
    # modified the plot_orbit_3d to pass extra parameters to the plt.plot() that actually adds the lines.
    tle.plot_orbit_3D(ax,xsat,label=sat.name)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))
for sat in gpsERS:
    t = tle.tledatenum(['2021-11-28 0:00:00', 24*60 ,1])
    xsat,vsat = tle.tle2vec(gpsERS,t,sat.name)
    xsate,vsate = crs.eci2ecef(t,xsat=xsat,vsat=vsat)
    tle.pltgroundtrack(xsate)



b.	Polar plot (skyplot), using the azimuth and zenith angles, for a station on the Earth (e.g. Delft).  Hint: You may use the function `skyplot` from the `tleplot`  module. 


In [ ]:
# to create a polar plot we need to define a location and find the look angles there
# we define a time array for this problem
t = tle.tledatenum(['2021-11-27 0:00:00', 1*24*60, 1])

# use location of delft in radians
delft_lat_rad = np.deg2rad(4.358)
delft_lon_rad = np.deg2rad(52.01144)

# convert lat/long to ecef location
delft_ecef = Re*np.array(
    [
        np.cos(delft_lat_rad)/np.cos(delft_lon_rad),  # X
        np.cos(delft_lat_rad)/np.sin(delft_lon_rad),  # Y
        np.sin(latlon[1]),  # Z
    ]
)
# print the delft coorindates to check them
print('Delft ecef coordinate: ', delft_ecef)
print(delft_ecef.shape)

# set up a function to get the look angles from delft


def lookangle_from_delft(t, sat_data_ecef):
    lookangle, flags = crs.satlookanglesp(t, sat_data_ecef, delft_ecef)
    return lookangle


plt.figure(figsize=(25, 25))
names = []

az_array = np.zeros((len(t), len(gpsERS)))
zen_array = np.zeros((len(t), len(gpsERS)))

for i, sat in enumerate(gpsERS):

    xsat, vsat = tle.tle2vec(gpsERS, t, sat.name)
    xsate, vsate = crs.eci2ecef(t, xsat=xsat, vsat=vsat)
    combined = np.concatenate((xsate, vsate), axis=1)
    las_array = lookangle_from_delft(t, combined)
    zen = las_array[:, 0]
    azi = las_array[:, 1]

    az_array[:, i] = azi
    zen_array[:, i] = zen
    names.append(sat.name)


tle.skyplot(t, az_array, zen_array, satnames=names, cutoff=10)



c.	Plot with the number of GPS satellite in view as function of time for the same ground station. Consider carefully how you define visibility and which value you use as elevation cut-off angle. Hint: As an intermediate step, create a 2D boolean array with an axis for satellite, and an axis for time, with True whenever a satellite is visible.

You may use a period of one day for these plots.

*Enter here your answers using markdown. Use one cell for each subquestion.*

## 3. Simulate and visualize direct signals

Simulate and visualize GNSS signals from the **upward looking omnidirectional** antenna on the ISS (direct signals). The opening angle of the upward looking antenna is 160 degrees (comparable to a 10 degree elevation cut-off angle). For this sub-task you have to use the simulated ISS and GPS positions from the previous sub-tasks. Questions to answer are:

a.	How many GPS satellites are on average in view from the upward looking antenna? Illustrate this by a plot with the number of GPS satellite in view as function of time from the ISS (as in 2c). How does this compare to a station on the Earth (e.g. Delft)?

b.	What is the average duration of GPS satellite visibility, in minutes from start to end for each pass, from the ISS? Include a plot that clearly illustrates this answer. Again, how does this compare to the a station on the Earth?

c.	Visualize the ISS, GPS and observations from the upward looking omnidirectional antenna in Google Earth using the `simplekml` Python module. You may get some inspiration from exercise 6 `CIE4604-M2-Example6_kml.ipynb` notebook. Make separate kml folders (within the same kmz file) for ISS, GPS and direct observations. For the report (Jupyter notebook), include a screenshot showing trajectories for one day, but the direct observations only for a single epoch.

You may use a period of one day for the plots. 

In [ ]:
# enter here your code and run the cell to produce the result, if you wish you can use more cells

*Enter here your answer using markdown. Use one cell for each sub-question. For question 3c do not forget to include the screenshot, you can do this with syntax like `![](screenshot.png)`*.

## 4. Simulate and visualize reflected signals


Simulate and visualize GNSS signals from the **downward looking** antenna on the ISS (reflected signals). The opening angle of the downward looking antenna is 40 degrees or 100 degrees.

a.	Make a function to compute the position of  the specular reflection points that are within the field of view (opening angle) of the down looking GPS antenna on board the ISS at a specified moment in time. 

Input for this function is:  the position of the ISS and  GPS satellite positions at a specific time (one epoch only), and the opening angle of the downward looking antenna (40 or 100 degrees). The output should be the position of the specular reflection points (zero, one or more) that fall within the opening angle. For the actual computation you may call the function `specularpoint` from the `specularpoint.py` module that is part of the `CIE4604_M2_Python.zip` distribution. This is also explained in more detail in exercise 8 `CIE4604_M2_Example8_sp.ipynb` notebook.

b.	Make a plot with the specular reflection points, ISS and GPS ground tracks in Python for a period of several minutes for two different values of the opening angle.

c.	Visualize the specular reflection points and reflected signals in Google Earth for two different values of the opening angle (40 degrees and 100 degrees), and add this to the kmz file of question 3c. Make a new kml folder (within the same kmz file) for each opening angle. 

Include in your report a screenshot of Google Earth showing the ISS, GPS and specular reflection point trajectories for a period of several minutes, while showing the direct and reflected signals only for a single epoch.

In [ ]:
# enter here your code and run the cell to produce the result, if you wish you can use more cells

*Enter here your answer using markdown. Use one cell for each sub-question. For question 3c do not forget to include the screenshot, you can do this with syntax like `![](screenshot.png)`*.

# Deliverables

The deliverables for this assignment are 

1.	Pdf of your Jupyter notebook or short report (pdf)  with a description of the GNSS-R simulator, plots of the results and answers to the questions.
2.	KMZ file (for display in Google Earth), one kmz file only, for question 3c and 4c combined.
3.	Jupyter notebook or zip file with the code.

The above deliverables have to be submitted electronically  through Brightspace.


[End of this Jupyter notebook]